# Capstone Project Module 3 – Machine Learning  
## Saudi Arabia Used Cars Price Prediction

Notebook ini merupakan dokumentasi lengkap pengerjaan **Capstone Project Module 3 (Machine Learning)**  
dengan studi kasus **prediksi harga mobil bekas di Arab Saudi**.

Struktur notebook:

1. Business Understanding  
2. Data Understanding  
3. Exploratory Data Analysis (EDA) – sebelum cleaning  
4. Data Cleaning & Perbandingan Sebelum vs Sesudah  
5. EDA Lanjutan – setelah cleaning  
6. Feature Engineering & Preprocessing  
7. Modeling (Baseline & Final Model)  
8. Evaluation & Interpretation  
9. Model Saving  
10. Conclusion & Recommendation


In [ ]:
# Import library utama
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

pd.set_option('display.max_columns', None)

## 1. Business Understanding

### 1.1 Latar Belakang

Misalkan ada sebuah platform marketplace mobil bekas bernama **UsedCarsKSA** di Arab Saudi.  
Platform ini menghubungkan penjual (individu maupun dealer) dengan calon pembeli mobil bekas.

**Sumber revenue utama platform:**
- Komisi transaksi penjualan mobil.
- Biaya iklan premium / highlight listing.
- Layanan tambahan (asuransi, pembiayaan, inspeksi, dll).

Saat ini, penentuan harga mobil bekas di platform masih sangat bergantung pada feeling dan pengalaman penjual.  
Sebagian besar penjual akan:

- Membandingkan secara manual dengan beberapa listing serupa.  
- Menentukan harga berdasarkan perkiraan pribadi.  
- Bertanya ke teman/kenalan atau bengkel.

Cara ini membuat harga menjadi **tidak konsisten** di seluruh platform.

---

### 1.2 Permasalahan yang Terjadi

Beberapa masalah yang muncul akibat penentuan harga yang tidak berbasis data:

- **Overpriced** – Harga terlalu tinggi → mobil sulit terjual, listing mengendap berminggu-minggu.  
- **Underpriced** – Harga terlalu rendah → mobil cepat laku tetapi penjual kehilangan potensi profit, dan platform kehilangan potensi komisi.  
- **Buyer tidak punya referensi jelas** apakah harga yang ia lihat itu wajar atau tidak.  

Dampak bisnis:

- Conversion rate menurun.  
- Time-to-sell menjadi lama.  
- User experience buruk dan mengurangi kepercayaan terhadap marketplace.

---

### 1.3 Mengapa Program Ini Diperlukan?

Marketplace mobil bekas memiliki variasi harga yang sangat tinggi karena penjual menentukan harga berdasarkan intuisi, bukan data.  
Ketidakkonsistenan harga menyebabkan masalah seperti:

- Listing overpriced → mobil sulit terjual → tidak ada komisi.  
- Listing underpriced → penjual dan platform kehilangan potensi revenue.  
- Buyer kehilangan kepercayaan karena sulit menilai fairness harga.  

Program prediksi harga berbasis Machine Learning diperlukan untuk:

- Menyediakan **estimasi harga yang objektif dan konsisten** berdasarkan data historis.  
- Membantu penjual menentukan harga optimal untuk mempercepat penjualan.  
- Memberikan indikator fairness bagi pembeli (misalnya: *good deal* / *fair* / *overpriced*).  
- Meningkatkan conversion rate dan revenue perusahaan.  

Dengan model ini, platform dapat memberikan *price guidance* secara real-time ketika seller membuat listing,
sehingga pengalaman penjual dan pembeli menjadi jauh lebih baik.

---

### 1.4 Problem Statement

> Bagaimana cara membantu penjual dan tim pricing untuk menentukan **harga jual mobil bekas yang wajar (fair market price)** berdasarkan data historis listing yang sudah ada?

Pertanyaan turunan:

- Seberapa baik kita bisa memprediksi `Price` berdasarkan fitur seperti `Year`, `Mileage`, `Make`, `Type`, dan lainnya?  
- Fitur apa yang paling memengaruhi harga mobil bekas?  

---

### 1.5 Tujuan Proyek

- Membangun model **Machine Learning (Regresi)** untuk memprediksi harga mobil bekas (`Price`).  
- Menghasilkan model dengan error serendah mungkin (diukur dengan **RMSE, MAE, R²**) dibanding baseline sederhana.  
- Mengidentifikasi **faktor-faktor utama** yang memengaruhi harga mobil bekas.  
- Menyediakan insight yang dapat digunakan tim bisnis untuk menyusun strategi pricing dan edukasi penjual.

---

### 1.6 Stakeholder

- **Tim Pricing / Business** – menggunakan output model sebagai referensi penentuan harga dasar dan strategi promosi.  
- **Penjual / Dealer** – mendapatkan rekomendasi harga saat membuat listing baru.  
- **Tim Product & Engineering** – mengintegrasikan model ke aplikasi (price suggestion).  
- **Manajemen** – mengukur dampak model terhadap kecepatan penjualan & revenue.

---

### 1.7 Analytical Approach

- Jenis masalah: **Regresi** (target numerik: `Price`).  
- Tools: Python, Pandas, NumPy, scikit-learn, Matplotlib.  
- Langkah utama:

  1. Memahami struktur dan konteks dataset.  
  2. Melakukan EDA untuk melihat pola dasar & outlier.  
  3. Membersihkan data dari nilai tidak valid.  
  4. Menyiapkan fitur (feature engineering, encoding, scaling).  
  5. Membagi data menjadi train & test set.  
  6. Membangun model baseline dan model utama.  
  7. Mengevaluasi performa model dengan RMSE, MAE, R².  
  8. Menginterpretasikan hasil dan menyusun rekomendasi bisnis.


## 2. Data Understanding

In [ ]:
# Load dataset
df = pd.read_csv('/mnt/data/data_saudi_used_cars.csv')
df.head()

Dari beberapa baris awal (`head()`), kita bisa melihat:

- Setiap baris merepresentasikan satu listing mobil bekas.  
- Setiap kolom adalah fitur yang mendeskripsikan mobil (brand, tahun, mileage, dll).  

Langkah ini penting untuk memastikan:

- Format data sudah sesuai ekspektasi.  
- Tidak ada kolom yang “aneh” atau tidak relevan di awal.


In [ ]:
# Ukuran data (baris, kolom)
df.shape

Output `shape` memberikan informasi:

- Jumlah baris → banyaknya listing mobil bekas.  
- Jumlah kolom → banyaknya fitur yang dimiliki.  

Semakin banyak baris:

- Model berpotensi belajar pola dengan lebih baik.  

Semakin banyak kolom:

- Informasi bisa lebih kaya, tapi kompleksitas preprocessing juga meningkat.


In [ ]:
# Informasi tipe data dan jumlah non-null
df.info()

Fungsi `info()` memberikan:

- Tipe data tiap kolom (`int64`, `float64`, `object`, dll).  
- Jumlah nilai non-null di tiap kolom.  

Dari sini kita bisa mengidentifikasi:

- Kolom mana yang **numerik** dan mana yang **kategorikal**.  
- Kolom mana yang mengandung **missing value** (jika `non-null` < jumlah baris).  

Informasi ini sangat penting untuk merencanakan:

- Teknik encoding untuk fitur kategorikal.  
- Treatment untuk missing value.


**Berdasarkan output `df.info()` pada dataset ini:**

- Total terdapat **5624 baris** data dan **11 kolom** fitur.  
- Semua kolom memiliki jumlah non-null yang sama dengan jumlah baris → **tidak ada missing value** pada dataset mentah.  
- Tipe data setiap kolom adalah:
  - `Type`, `Region`, `Make`, `Gear_Type`, `Origin`, `Options` → `object` (kategorikal).  
  - `Year`, `Mileage`, `Price` → `int64` (numerik).  
  - `Engine_Size` → `float64` (numerik kontinu).  
  - `Negotiable` → `bool` (True/False).  

Informasi ini mengonfirmasi bahwa secara struktur, dataset cukup rapi dan tidak memiliki nilai kosong,
sehingga fokus utama preprocessing akan lebih ke **pembersihan nilai tidak logis** (misalnya `Price = 0`)
dan penanganan tipe data kategorikal sebelum masuk ke tahap modeling.


In [ ]:
# Statistik deskriptif numerik
df.describe().T

`describe()` memberikan:

- `count` : jumlah data yang valid.  
- `mean` : rata-rata.  
- `std` : standar deviasi (sebaran data).  
- `min` & `max` : nilai minimum dan maksimum.  
- `25%`, `50%`, `75%` : kuartil.  

Dari sini kita bisa:

- Mendeteksi nilai `Price` yang sangat kecil (misalnya 0) yang kemungkinan tidak valid.  
- Melihat apakah `Mileage` memiliki nilai maksimum yang tidak realistis (indikasi outlier).  
- Memahami skala setiap variabel numerik.


**Contoh interpretasi untuk kolom `Price`:**

- `count` = 5624 → jumlah baris dengan nilai harga valid di dataset mentah.  
- `mean` ≈ 53074.06 → rata-rata harga mobil bekas sekitar 53074 Riyal.  
- `std` ≈ 70155.34 → standar deviasi cukup besar, menandakan sebaran harga yang sangat lebar.  
- `min` = 0 → terdapat harga **0 Riyal**, yang kemungkinan besar adalah nilai tidak valid.  
- `25%` = 0 dan `50%` = 36500 → bahkan kuartil 1 dan median masih 0, artinya **lebih dari 50% data memiliki Price = 0**.  
- `75%` = 72932 → 25% data teratas memiliki harga di atas nilai ini.  
- `max` = 850000 → harga maksimum mencapai 850000 Riyal, yang kemungkinan merepresentasikan mobil premium atau outlier harga tinggi.  

Dari sini terlihat jelas bahwa kolom `Price` mengandung banyak nilai 0 dan sebaran yang sangat lebar.
Hal ini menjadi alasan kuat kenapa **data cleaning pada `Price` wajib dilakukan** sebelum training model.


In [ ]:
# Cek jumlah missing value per kolom
df.isna().sum()

Output `isna().sum()` menunjukkan berapa banyak nilai yang hilang di setiap kolom.

Dari sini kita bisa:

- Menentukan fitur mana yang perlu **treatment khusus**.  
- Jika jumlah missing sangat besar di satu kolom, kita bisa mempertimbangkan untuk:

  - Menghapus kolom tersebut, atau  
  - Mengisi dengan strategi tertentu (imputasi).  

Langkah selanjutnya adalah melihat apakah missing value ini akan kita drop,
isi (impute), atau abaikan jika memang tidak terlalu penting.


Jika kita lihat output `df.isna().sum()`, seluruh kolom memiliki nilai **0** untuk jumlah missing value.

Artinya:

- Tidak ada satupun baris yang memiliki nilai `NaN` di kolom manapun.  
- Kita **tidak perlu melakukan imputasi** (mengisi nilai kosong) pada dataset ini.  

Dengan demikian, fokus cleaning berpindah dari penanganan missing value ke:

- Menghapus nilai yang tidak logis (misalnya `Price = 0`).  
- Mengurangi outlier ekstrem pada fitur numerik seperti `Mileage`.


## 3. Exploratory Data Analysis (EDA) – Sebelum Cleaning

### 3.1 Distribusi Target (Price)

In [ ]:
plt.figure(figsize=(7,4))
plt.hist(df['Price'], bins=60)
plt.title('Distribusi Harga (Price) – Sebelum Cleaning')
plt.xlabel('Price')
plt.ylabel('Frekuensi')
plt.show()

df['Price'].describe()

Histogram `Price` dipakai untuk melihat:

- Bentuk distribusi harga → apakah normal, skew, atau banyak outlier.  
- Adanya nilai 0 atau sangat kecil yang tidak masuk akal sebagai harga mobil.  

Dari grafik dan statistik:

- Jika terlihat banyak nilai di 0, besar kemungkinan itu adalah data tidak valid.  
- Nilai maksimum yang sangat besar juga bisa menjadi outlier.  

Temuan ini menjadi dasar kenapa kita perlu **data cleaning** sebelum modeling.


### 3.2 Distribusi Fitur Numerik

In [ ]:
numeric_cols = ['Year', 'Engine_Size', 'Mileage']

plt.figure(figsize=(14,3))
for i, col in enumerate(numeric_cols):
    plt.subplot(1, len(numeric_cols), i+1)
    plt.hist(df[col], bins=40)
    plt.title(col)
plt.tight_layout()
plt.show()

Histogram `Year`, `Engine_Size`, dan `Mileage` membantu kita memahami:

- Tahun produksi mana yang paling banyak muncul → apakah mobil relatif baru atau tua.  
- Kapasitas mesin yang dominan di dataset.  
- Distribusi `Mileage` untuk melihat apakah banyak mobil dengan jarak tempuh ekstrem.  

Jika `Mileage` memiliki ekor yang sangat panjang (banyak nilai besar),  
ini mengindikasikan adanya outlier yang harus diperhatikan pada proses cleaning.


### 3.3 Distribusi Fitur Kategorikal

In [ ]:
cat_cols_to_view = ['Make', 'Region', 'Gear_Type', 'Options', 'Origin']

for col in cat_cols_to_view:
    print(f"\nKolom: {col}")
    print(df[col].value_counts().head(10))

Dengan `value_counts()` kita bisa melihat 10 kategori terbanyak di setiap kolom kategorikal.

Insight yang bisa diambil:

- Brand (`Make`) apa yang paling populer.  
- Region mana yang memiliki listing paling banyak.  
- Sebaran `Gear_Type` (Automatic vs Manual).  

Ini membantu kita memahami komposisi data dan memikirkan 
apakah perlu perlakuan khusus untuk kategori yang sangat jarang.


In [ ]:
top_makes = df['Make'].value_counts().head(10)
plt.figure(figsize=(8,4))
plt.bar(top_makes.index, top_makes.values)
plt.xticks(rotation=45)
plt.title("Top 10 Brand (Make) berdasarkan jumlah listing")
plt.ylabel("Count")
plt.show()

Grafik ini menunjukkan 10 brand dengan jumlah listing terbanyak.

Interpretasi:

- Brand dengan jumlah listing tinggi akan sangat berkontribusi pada pembelajaran model.  
- Untuk brand dengan listing sedikit, pola harga mungkin lebih sulit dipelajari secara akurat.

Informasi ini juga bisa dipakai tim bisnis untuk tahu brand apa yang paling laris di platform.


In [ ]:
gear_counts = df['Gear_Type'].value_counts()
plt.figure(figsize=(5,4))
plt.bar(gear_counts.index, gear_counts.values)
plt.title("Distribusi Gear_Type")
plt.ylabel("Count")
plt.show()

Distribusi `Gear_Type` menjawab pertanyaan:

- Apakah pengguna di Arab Saudi cenderung menggunakan mobil Automatic atau Manual?  

Jika mayoritas adalah Automatic, maka:

- Model akan lebih banyak belajar dari pola mobil bertransmisi Automatic.  
- Ini juga memberi insight preferensi pasar untuk keperluan marketing.


### 3.4 Hubungan Fitur dengan Price

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(df['Year'], df['Price'], alpha=0.3)
plt.title('Year vs Price (Sebelum Cleaning)')
plt.xlabel('Year')
plt.ylabel('Price')
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(df['Mileage'], df['Price'], alpha=0.3)
plt.title('Mileage vs Price (Sebelum Cleaning)')
plt.xlabel('Mileage')
plt.ylabel('Price')
plt.show()

Dua scatter plot di atas memberi gambaran:

- **Year vs Price** → mobil dengan tahun produksi lebih baru cenderung memiliki harga lebih tinggi.  
- **Mileage vs Price** → mobil dengan mileage lebih tinggi cenderung memiliki harga lebih rendah.  

Walaupun ada noise, pola ini konsisten dengan teori depresiasi mobil bekas.


## 4. Data Cleaning & Perbandingan Sebelum vs Sesudah

Langkah-langkah cleaning utama:

1. Menghapus baris dengan `Price <= 0` (harga tidak valid).  
2. Menghapus outlier ekstrem pada `Mileage` (misalnya > 1.000.000 km).  
3. Menghapus baris duplikat untuk menghindari bias.  

Tujuannya adalah meningkatkan kualitas data sehingga model tidak belajar dari nilai yang tidak realistis.


In [ ]:
df_clean = df.copy()

# 1. Hapus Price tidak valid
df_clean = df_clean[df_clean['Price'] > 0]

# 2. Batasi Mileage maksimal 1.000.000 km
df_clean = df_clean[df_clean['Mileage'] < 1_000_000]

# 3. Drop duplikat
df_clean = df_clean.drop_duplicates()

df_clean.shape

Setelah cleaning, `shape` menunjukkan berapa banyak data berkualitas yang tersisa.  
Jika proporsi data yang hilang tidak terlalu besar, maka cleaning ini justru meningkatkan kualitas model.


In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.hist(df['Price'], bins=60)
plt.title("Price – Sebelum Cleaning")

plt.subplot(1,2,2)
plt.hist(df_clean['Price'], bins=60)
plt.title("Price – Setelah Cleaning")

plt.tight_layout()
plt.show()

Perbandingan histogram `Price`:

- Sebelum cleaning → banyak nilai 0 atau nilai ekstrim.  
- Setelah cleaning → distribusi harga lebih realistis dan smooth.  

Ini memastikan model tidak “tertipu” oleh harga 0 atau nilai ekstrem yang salah input.


In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.hist(df['Mileage'], bins=60)
plt.title("Mileage – Sebelum Cleaning")

plt.subplot(1,2,2)
plt.hist(df_clean['Mileage'], bins=60)
plt.title("Mileage – Setelah Cleaning")

plt.tight_layout()
plt.show()

Perbandingan distribusi `Mileage`:

- Sebelum cleaning → ekor distribusi sangat panjang berisi nilai mileage yang sangat besar.  
- Setelah cleaning → distribusi menjadi lebih wajar dan aman untuk proses training.  

Outlier yang terlalu ekstrem bisa membuat model menjadi tidak stabil,
sehingga mengurangi kemampuan generalisasi.


## 5. EDA Lanjutan – Setelah Cleaning

In [ ]:
df_clean.describe().T

`describe()` setelah cleaning memastikan bahwa:

- Nilai minimum dan maksimum `Price` dan `Mileage` sekarang wajar.  
- Nilai rata-rata dan kuartil menjadi lebih representatif terhadap populasi data yang sehat.  

Ini menjadi fondasi yang baik sebelum masuk ke tahap modeling.


In [ ]:
df_clean['log_price'] = np.log1p(df_clean['Price'])

plt.figure(figsize=(7,4))
plt.hist(df_clean['log_price'], bins=60)
plt.title("Distribusi log(Price + 1) – Setelah Cleaning")
plt.xlabel("log(Price + 1)")
plt.ylabel("Frekuensi")
plt.show()

Transformasi log (`log(Price+1)`) membuat distribusi harga lebih simetris dan tidak terlalu skew.  
Meskipun model tetap memprediksi `Price` asli, eksplorasi log-price membantu memahami pola data
dan bisa menjadi opsi untuk eksperimen model lain.


In [ ]:
corr = df_clean[['Year','Engine_Size','Mileage','Price']].corr()

plt.figure(figsize=(5,4))
plt.imshow(corr, cmap='coolwarm', interpolation='nearest')
plt.colorbar()
plt.xticks(range(len(corr)), corr.columns, rotation=45)
plt.yticks(range(len(corr)), corr.columns)
plt.title("Correlation Matrix – Numeric Features")
plt.tight_layout()
plt.show()

corr

Matriks korelasi memberi gambaran hubungan linear antar variabel numerik.

Insight umum:

- `Year` berkorelasi positif dengan `Price` → mobil baru cenderung lebih mahal.  
- `Mileage` berkorelasi negatif dengan `Price` → jarak tempuh tinggi menurunkan harga.  
- `Engine_Size` biasanya berkorelasi positif dengan `Price` → mesin lebih besar, harga lebih mahal.  

Walaupun Random Forest tidak bergantung pada korelasi linear,
analisis ini membantu memvalidasi intuisi domain.


## 6. Feature Engineering & Preprocessing

Langkah-langkah:

1. Menambahkan fitur `Car_Age` untuk mewakili umur mobil.  
2. Memisahkan fitur (`X`) dan target (`y`).  
3. Menentukan kolom numerik dan kategorikal.  
4. Menyiapkan `ColumnTransformer` untuk scaling & encoding.  
5. Melakukan train–test split.


In [ ]:
# Feature Engineering: Car_Age
current_year = df_clean['Year'].max()
df_clean['Car_Age'] = current_year - df_clean['Year']

# Pisahkan fitur & target
X = df_clean.drop(['Price','log_price'], axis=1)
y = df_clean['Price']

# Identifikasi kolom numerik & kategorikal
num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object','bool']).columns.tolist()

num_cols, cat_cols

Fitur baru **Car_Age** merepresentasikan umur mobil secara eksplisit:

- Semakin tinggi `Car_Age`, semakin tua mobil, biasanya harga makin turun.  

Pemisahan `num_cols` dan `cat_cols` diperlukan untuk:

- Mengaplikasikan scaling hanya pada fitur numerik.  
- Menerapkan one-hot encoding pada fitur kategorikal.


In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train.shape, X_test.shape

Train-test split:

- 80% data → training (model belajar pola).  
- 20% data → testing (mengevaluasi kemampuan generalisasi).  

Hal ini mencegah **overfitting**, yaitu kondisi di mana model sangat bagus di data training
tetapi buruk di data baru.


In [ ]:
# Preprocessing: scaling & one-hot encoding
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ]
)

preprocess

`ColumnTransformer` menggabungkan seluruh proses preprocessing:

- Fitur numerik → distandarisasi dengan `StandardScaler`.  
- Fitur kategorikal → diubah menjadi dummy variabel dengan `OneHotEncoder`.  

Dengan memasukkan `preprocess` ke dalam `Pipeline` bersama model,
kita memastikan preprocessing dan modeling dilakukan secara konsisten
baik saat training maupun saat melakukan prediksi data baru.


## 7. Modeling

### 7.1 Baseline Model – Dummy Regressor

In [ ]:
baseline_model = DummyRegressor(strategy='median')

baseline_pipeline = Pipeline(steps=[
    ('preprocess', preprocess),
    ('model', baseline_model)
])

baseline_pipeline.fit(X_train, y_train)
y_pred_baseline = baseline_pipeline.predict(X_test)

rmse_baseline = mean_squared_error(y_test, y_pred_baseline) ** 0.5
mae_baseline = mean_absolute_error(y_test, y_pred_baseline)
r2_baseline = r2_score(y_test, y_pred_baseline)

print(f"Baseline RMSE: {rmse_baseline:,.2f}")
print(f"Baseline MAE : {mae_baseline:,.2f}")
print(f"Baseline R2  : {r2_baseline:.3f}")

**DummyRegressor (median)** adalah baseline model yang **tidak belajar pola apapun**.  
Model ini hanya memprediksi median `Price` dari data training untuk semua kasus.

Tiga metrik yang digunakan:

1. **RMSE (Root Mean Squared Error)**  

\[ RMSE = \sqrt{\frac{1}{n} \sum (y_i - \hat{y_i})^2} \]

- Mengukur rata-rata kesalahan prediksi dalam **satuan harga**.  
- Error besar diberi penalti lebih tinggi karena dikuadratkan.  

2. **MAE (Mean Absolute Error)**  

\[ MAE = \frac{1}{n} \sum |y_i - \hat{y_i}| \]

- Mengukur rata-rata selisih absolut antara nilai aktual dan nilai prediksi.  
- Lebih mudah diinterpretasikan, misalnya: “model rata-rata meleset 15.000 Riyal”.  

3. **R² (Coefficient of Determination)**  

\[ R^2 = 1 - \frac{SS_{res}}{SS_{tot}} \]

- Mengukur seberapa besar variasi `Price` yang dapat dijelaskan oleh model.  
- Nilai 1 → model sangat baik, 0 → sekelas menebak rata-rata, nilai negatif → lebih buruk dari menebak rata-rata.  

Hasil baseline ini menjadi **tolok ukur minimum**.  
Model Machine Learning yang baik harus memiliki:

- RMSE & MAE **lebih kecil** dari baseline.  
- R² **lebih tinggi** (mendekati 1).


Berdasarkan hasil perhitungan pada dataset ini:

- **Baseline RMSE ≈ 64,991.26**  
  → Rata-rata kesalahan model baseline sekitar 64,991 Riyal dalam satuan harga.  

- **Baseline MAE ≈ 40,436.79**  
  → Secara rata-rata absolut, tebakan median ini meleset sekitar 40,437 Riyal dari harga sebenarnya.  

- **Baseline R² ≈ -0.072**  
  → Nilai R² negatif (~-0.072) menunjukkan bahwa model baseline **lebih buruk daripada sekadar menebak rata-rata**.
  
Interpretasinya:

- DummyRegressor ini belum memanfaatkan informasi fitur apapun, sehingga performanya buruk.  
- Hasil ini menjadi **benchmark minimum** — model Machine Learning yang baik harus bisa mengalahkan angka-angka ini.


### 7.2 Random Forest Regressor (Final Model)

In [ ]:
rf_model = RandomForestRegressor(
    n_estimators=180,
    random_state=42,
    n_jobs=-1
)

rf_pipeline = Pipeline(steps=[
    ('preprocess', preprocess),
    ('model', rf_model)
])

rf_pipeline.fit(X_train, y_train)
y_pred_rf = rf_pipeline.predict(X_test)

rmse_rf = mean_squared_error(y_test, y_pred_rf) ** 0.5
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"Random Forest RMSE: {rmse_rf:,.2f}")
print(f"Random Forest MAE : {mae_rf:,.2f}")
print(f"Random Forest R2  : {r2_rf:.3f}")

**Random Forest Regressor** dipilih sebagai model utama karena:

- Dapat menangkap hubungan non-linear antara fitur dan harga.  
- Relatif robust terhadap outlier dan noise.  
- Bekerja sangat baik pada data tabular seperti dataset ini.  

Interpretasi metrik Random Forest vs baseline:

- Jika **RMSE & MAE Random Forest lebih kecil** dibanding baseline → rata-rata kesalahan prediksi menurun.  
- Jika **R² Random Forest jauh lebih tinggi** → model mampu menjelaskan variasi harga dengan lebih baik.  

Secara bisnis, penurunan error berarti:

- Rekomendasi harga untuk seller lebih dekat dengan harga pasar.  
- Risiko salah harga (terlalu murah/mahal) berkurang.  
- Buyer lebih percaya bahwa harga di platform cukup fair.


Pada model **Random Forest** yang dilatih pada dataset ini, diperoleh hasil:

- **Random Forest RMSE ≈ 25,418.97**  
  → Rata-rata kesalahan dalam satuan harga turun menjadi sekitar 25,419 Riyal.  

- **Random Forest MAE ≈ 13,879.11**  
  → Selisih absolut rata-rata antara harga aktual dan prediksi sekitar 13,879 Riyal.  
  Angka ini jauh lebih kecil dibanding baseline.  

- **Random Forest R² ≈ 0.836**  
  → Nilai R² sekitar 0.836 berarti **sekitar 83,6% variasi harga** dapat dijelaskan oleh model.  

Dibandingkan dengan baseline:

- RMSE turun dari 64,991 → 25,419 Riyal.  
- MAE turun dari 40,437 → 13,879 Riyal.  
- R² naik dari -0.072 → 0.836.  

Secara praktis, ini menunjukkan peningkatan performa yang sangat signifikan — 
model kini mampu memberikan rekomendasi harga yang jauh lebih dekat dengan harga pasar sebenarnya.


## 8. Evaluation & Interpretation

In [ ]:
print("=== Baseline (Dummy Median) ===")
print(f"RMSE: {rmse_baseline:,.2f}")
print(f"MAE : {mae_baseline:,.2f}")
print(f"R2  : {r2_baseline:.3f}\n")

print("=== Random Forest ===")
print(f"RMSE: {rmse_rf:,.2f}")
print(f"MAE : {mae_rf:,.2f}")
print(f"R2  : {r2_rf:.3f}")

Dari perbandingan di atas:

- **RMSE & MAE Random Forest lebih kecil** → rata-rata kesalahan prediksi berkurang secara signifikan.  
- **R² Random Forest lebih tinggi** → model menjelaskan porsi variasi harga yang jauh lebih besar.  

Secara bisnis, hal ini berarti:

- Seller mendapatkan rekomendasi harga yang lebih akurat.  
- Buyer melihat harga yang lebih wajar.  
- Platform dapat mengurangi jumlah listing yang terlalu mahal atau terlalu murah.


### 8.1 Actual vs Predicted

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred_rf, alpha=0.3)
plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")
plt.title("Actual vs Predicted Price (Random Forest)")

min_val = min(y_test.min(), y_pred_rf.min())
max_val = max(y_test.max(), y_pred_rf.max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--')
plt.show()

Grafik ini membandingkan harga aktual vs prediksi:

- Setiap titik adalah satu mobil di test set.  
- Garis merah adalah garis ideal (prediksi = aktual).  

Semakin banyak titik yang berada dekat garis merah, semakin baik kualitas prediksi.  
Jika banyak titik jauh di atas garis → model sering overestimate.  
Jika banyak titik jauh di bawah garis → model sering underestimate.


### 8.2 Distribusi Residual

In [ ]:
residuals = y_test - y_pred_rf

plt.figure(figsize=(7,4))
plt.hist(residuals, bins=60)
plt.title("Distribusi Residual (y_true - y_pred)")
plt.xlabel("Residual")
plt.ylabel("Frekuensi")
plt.show()

Residual = selisih antara harga aktual dan prediksi (`y_true - y_pred`).  

- Jika distribusi residual simetris di sekitar 0 → model tidak terlalu bias.  
- Jika cenderung positif/negatif saja → model cenderung terlalu mahal/telalu murah.  

Tujuan ideal: residual menyebar di sekitar 0 tanpa pola tertentu,
yang artinya error model bersifat acak dan bukan karena pola yang terlewat.


### 8.3 Feature Importance

In [ ]:
rf = rf_pipeline.named_steps['model']
ohe = rf_pipeline.named_steps['preprocess'].named_transformers_['cat'].named_steps['onehot']

cat_feature_names = ohe.get_feature_names_out(cat_cols)
all_feature_names = np.concatenate([num_cols, cat_feature_names])

importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]

top_n = 15

plt.figure(figsize=(8,6))
plt.barh(range(top_n), importances[indices][:top_n][::-1])
plt.yticks(range(top_n), all_feature_names[indices][:top_n][::-1])
plt.title("Top 15 Feature Importances (Random Forest)")
plt.xlabel("Importance")
plt.tight_layout()
plt.show()

Grafik **feature importance** menjelaskan fitur mana yang paling berpengaruh terhadap prediksi harga.

Biasanya fitur yang muncul di peringkat teratas adalah:

- `Year` / `Car_Age` → umur mobil.  
- `Mileage` → jarak tempuh.  
- `Engine_Size` → kapasitas mesin.  
- Beberapa kategori brand (`Make_...`) dan tipe (`Type_...`).  
- Level fitur (`Options_Full`, dll).  

Insight ini bisa digunakan untuk:

- Edukasi seller: faktor apa yang paling menentukan harga mobil bekas.  
- Bahan konten marketing & artikel blog di platform.


## 9. Model Saving

In [ ]:
# Simpan model (pipeline lengkap)
final_model = rf_pipeline
joblib.dump(final_model, 'model_saudi_used_cars.pkl')

'model_saudi_used_cars.pkl saved'

Model yang disimpan (`model_saudi_used_cars.pkl`) sudah mencakup:

- Langkah preprocessing (scaling + encoding).  
- Model Random Forest yang sudah dilatih.  

Untuk memprediksi data baru, langkahnya:

1. Load model dengan `joblib.load('model_saudi_used_cars.pkl')`.  
2. Siapkan DataFrame baru dengan kolom yang sama seperti `X`.  
3. Panggil `model.predict(data_baru)` untuk mendapatkan estimasi harga.


## 10. Conclusion & Recommendation

### 10.1 Conclusion

- Model **Random Forest Regressor** berhasil mengurangi error secara signifikan dibanding baseline Dummy Regressor.  
- RMSE dan MAE turun, R² naik → model mampu menjelaskan sebagian besar variasi harga dalam data.  
- Fitur yang paling berpengaruh terhadap harga adalah:

  - Umur mobil (Year / Car_Age).  
  - Mileage (jarak tempuh).  
  - Kapasitas mesin (Engine_Size).  
  - Brand dan tipe mobil.  
  - Level fitur (Options).  

Pola ini konsisten dengan logika pasar mobil bekas: mobil yang lebih baru, mileage rendah,
brand premium, dan fitur lengkap cenderung memiliki harga lebih mahal.

---

### 10.2 Business Recommendation

1. **Price Recommendation di Halaman Posting**  

   - Saat seller mengisi detail mobil (brand, tahun, mileage, options, dll.), sistem dapat memberikan:
     - Estimasi harga utama.  
     - Rentang harga rekomendasi (misalnya 65.000–75.000 Riyal).  
   - Seller dapat melihat apakah harga yang ia masukkan termasuk *below market*, *fair*, atau *above market*.

2. **Fair Price Indicator untuk Buyer**  

   - Di halaman detail mobil, tampilkan label seperti *Good Deal*, *Fair Price*, dan *Overpriced* berdasarkan perbandingan
     harga listing vs estimasi model.  
   - Hal ini meningkatkan kepercayaan buyer dan membantu mereka mengambil keputusan lebih cepat.

3. **Edukasi Seller Menggunakan Feature Importance**  

   - Buat konten edukatif untuk menjelaskan faktor penentu harga utama (umur, mileage, kapasitas mesin, fitur).  
   - Seller akan lebih memahami kenapa sistem merekomendasikan harga tertentu,
     sehingga lebih mudah menerima rekomendasi tersebut.

4. **Segmentasi & Strategi Promosi**  

   - Analisis rata-rata harga per brand/region dapat digunakan untuk:  
     - Menentukan segmen premium vs mass-market.  
     - Menentukan campaign promosi yang lebih tepat sasaran.  

---

### 10.3 Model Limitation & Future Work

**Limitasi:**

- Tidak ada informasi detail kondisi fisik mobil (riwayat servis, kecelakaan, modifikasi).  
- Lokasi masih dalam bentuk region, belum sampai area spesifik.  
- Faktor eksternal seperti tren ekonomi atau perubahan harga BBM belum dipertimbangkan.

**Future Work:**

- Menambah fitur baru seperti skor inspeksi kendaraan, jumlah pemilik, jenis bahan bakar, dan tipe body.  
- Mencoba algoritma lain (XGBoost, CatBoost, Gradient Boosting) dan melakukan hyperparameter tuning.  
- Melakukan deployment dan monitoring berkala untuk mendeteksi data drift dan penurunan performa model.  

Dengan pengembangan lanjutan, model ini berpotensi menjadi komponen inti dari sistem **dynamic pricing**
di marketplace mobil bekas dan memberikan nilai tambah yang besar bagi user maupun perusahaan.
